In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Предположим, у нас есть следующие датафреймы:
# interactions - все взаимодействия пользователей с объектами
# items - метаданные объектов
# users - метаданные пользователей (если есть)

# Примерная структура данных:
print("Пример данных взаимодействий:")
print("columns: ['user_id', 'item_id', 'interaction_type', 'rating', 'timestamp']")
print("\nПример метаданных объектов:")
print("columns: ['item_id', 'category', 'brand', 'price', 'created_at', 'title', 'description']")

Пример данных взаимодействий:
columns: ['user_id', 'item_id', 'interaction_type', 'rating', 'timestamp']

Пример метаданных объектов:
columns: ['item_id', 'category', 'brand', 'price', 'created_at', 'title', 'description']


In [2]:
def create_item_popularity_features(interactions_df, items_meta_df, current_date=None):
    """
    Создает признаки популярности объектов.

    Args:
        interactions_df: DataFrame с колонками ['item_id', 'timestamp', 'interaction_type']
        items_meta_df: DataFrame с метаданными объектов
        current_date: Дата, на которую считаем признаки (для валидации/теста)

    Returns:
        DataFrame с признаками для каждого item_id
    """
    if current_date is None:
        current_date = interactions_df['timestamp'].max()
    else:
        current_date = pd.to_datetime(current_date)

    # Копируем датафрейм, чтобы не менять оригинал
    interactions = interactions_df.copy()
    interactions['timestamp'] = pd.to_datetime(interactions['timestamp'])

    # Создаем пустой датафрейм для результатов
    item_features = pd.DataFrame()
    item_features['item_id'] = interactions['item_id'].unique()

    # Базовые даты для временных окон
    one_day_ago = current_date - timedelta(days=1)
    seven_days_ago = current_date - timedelta(days=7)
    thirty_days_ago = current_date - timedelta(days=30)
    fourteen_days_ago = current_date - timedelta(days=14)

    # 1. Популярность за всё время
    total_popularity = interactions.groupby('item_id').size().reset_index(name='item_popularity_total')
    item_features = item_features.merge(total_popularity, on='item_id', how='left')
    item_features['item_popularity_total'] = item_features['item_popularity_total'].fillna(0).astype(int)

    # 2. Популярность за временные окна
    # Последние 30 дней
    recent_30d = interactions[interactions['timestamp'] >= thirty_days_ago]
    pop_30d = recent_30d.groupby('item_id').size().reset_index(name='item_popularity_30d')
    item_features = item_features.merge(pop_30d, on='item_id', how='left')
    item_features['item_popularity_30d'] = item_features['item_popularity_30d'].fillna(0).astype(int)

    # Последние 7 дней
    recent_7d = interactions[interactions['timestamp'] >= seven_days_ago]
    pop_7d = recent_7d.groupby('item_id').size().reset_index(name='item_popularity_7d')
    item_features = item_features.merge(pop_7d, on='item_id', how='left')
    item_features['item_popularity_7d'] = item_features['item_popularity_7d'].fillna(0).astype(int)

    # Последние 1 день
    recent_1d = interactions[interactions['timestamp'] >= one_day_ago]
    pop_1d = recent_1d.groupby('item_id').size().reset_index(name='item_popularity_1d')
    item_features = item_features.merge(pop_1d, on='item_id', how='left')
    item_features['item_popularity_1d'] = item_features['item_popularity_1d'].fillna(0).astype(int)

    # 3. Тренд популярности (отношение последней недели к предыдущей)
    week1 = interactions[(interactions['timestamp'] >= seven_days_ago) &
                        (interactions['timestamp'] < one_day_ago)]
    week2 = interactions[(interactions['timestamp'] >= fourteen_days_ago) &
                        (interactions['timestamp'] < seven_days_ago)]

    pop_week1 = week1.groupby('item_id').size().reset_index(name='pop_week1')
    pop_week2 = week2.groupby('item_id').size().reset_index(name='pop_week2')

    # Объединяем и считаем тренд
    trend_df = pop_week1.merge(pop_week2, on='item_id', how='outer').fillna(0.1)  # 0.1 чтобы избежать деления на 0
    trend_df['item_popularity_trend'] = (trend_df['pop_week1'] + 1) / (trend_df['pop_week2'] + 1)

    item_features = item_features.merge(trend_df[['item_id', 'item_popularity_trend']],
                                       on='item_id', how='left')
    item_features['item_popularity_trend'] = item_features['item_popularity_trend'].fillna(1.0)

    # 4. Стандартное отклонение популярности по дням
    # Группируем по дням для каждого объекта
    interactions['date'] = interactions['timestamp'].dt.date

    # Берем данные за последние 30 дней для стабильности
    recent_for_std = interactions[interactions['timestamp'] >= thirty_days_ago]
    daily_popularity = recent_for_std.groupby(['item_id', 'date']).size().reset_index(name='daily_count')

    # Если у объекта мало дней с взаимодействиями, заполняем нулями
    all_dates = pd.date_range(start=thirty_days_ago.date(), end=current_date.date(), freq='D')
    all_items = interactions['item_id'].unique()

    # Создаем полную сетку и заполняем нулями отсутствующие дни
    idx = pd.MultiIndex.from_product([all_items, all_dates], names=['item_id', 'date'])
    full_grid = pd.DataFrame(index=idx).reset_index()
    full_grid['date'] = pd.to_datetime(full_grid['date'])

    daily_popularity['date'] = pd.to_datetime(daily_popularity['date'])
    full_popularity = pd.merge(full_grid, daily_popularity,
                              on=['item_id', 'date'], how='left')
    full_popularity['daily_count'] = full_popularity['daily_count'].fillna(0)

    # Считаем стандартное отклонение
    popularity_std = full_popularity.groupby('item_id')['daily_count'].std().reset_index(name='item_popularity_std')
    popularity_std['item_popularity_std'] = popularity_std['item_popularity_std'].fillna(0)

    item_features = item_features.merge(popularity_std, on='item_id', how='left')

    # 5. Временные признаки объекта
    if 'created_at' in items_meta_df.columns:
        items_meta_df['created_at'] = pd.to_datetime(items_meta_df['created_at'])

        # Возраст объекта в днях
        items_meta_df['item_age_days'] = (current_date - items_meta_df['created_at']).dt.days

        # Флаг нового объекта
        items_meta_df['is_new_item'] = (items_meta_df['item_age_days'] <= 7).astype(int)

        item_features = item_features.merge(
            items_meta_df[['item_id', 'item_age_days', 'is_new_item']],
            on='item_id', how='left'
        )

    # 6. Days since last interaction
    last_interaction = interactions.groupby('item_id')['timestamp'].max().reset_index()
    last_interaction['days_since_last_interaction'] = (
        current_date - last_interaction['timestamp']
    ).dt.days

    item_features = item_features.merge(
        last_interaction[['item_id', 'days_since_last_interaction']],
        on='item_id', how='left'
    )
    item_features['days_since_last_interaction'] = item_features['days_since_last_interaction'].fillna(999)

    return item_features

In [8]:
def create_item_interaction_stats(interactions_df, items_meta_df=None):
    """
    Создает статистики по взаимодействиям с объектами.

    Args:
        interactions_df: DataFrame с колонками ['item_id', 'interaction_type', 'rating']
                         interaction_type может быть: 'click', 'purchase', 'view', 'add_to_cart', etc.
                         rating: числовой рейтинг (1-5)
    """
    interactions = interactions_df.copy()
    item_stats = pd.DataFrame()
    item_stats['item_id'] = interactions['item_id'].unique()

    # 1. Средний рейтинг
    if 'rating' in interactions.columns:
        avg_rating = interactions.groupby('item_id')['rating'].mean().reset_index(name='item_avg_rating')
        rating_count = interactions.groupby('item_id')['rating'].count().reset_index(name='item_rating_count')

        item_stats = item_stats.merge(avg_rating, on='item_id', how='left')
        item_stats = item_stats.merge(rating_count, on='item_id', how='left')

        # Распределение оценок
        for rating_val in [1, 2, 3, 4, 5]:
            rating_mask = interactions['rating'] == rating_val
            rating_counts = interactions[rating_mask].groupby('item_id').size().reset_index(name=f'item_rating_{rating_val}_stars')
            item_stats = item_stats.merge(rating_counts, on='item_id', how='left')
            item_stats[f'item_rating_{rating_val}_stars'] = item_stats[f'item_rating_{rating_val}_stars'].fillna(0)

        # Процент 5-звездочных оценок
        item_stats['item_pct_5_stars'] = np.where(
            item_stats['item_rating_count'] > 0,
            item_stats['item_rating_5_stars'] / item_stats['item_rating_count'],
            0
        )

    # 2. CTR и конверсии (если есть данные о показах)
    if 'interaction_type' in interactions.columns:
        # Считаем разные типы взаимодействий
        interaction_counts = interactions.groupby(['item_id', 'interaction_type']).size().unstack(fill_value=0)

        # CTR (клики / показы)
        if 'click' in interaction_counts.columns and 'view' in interaction_counts.columns:
            item_stats['item_ctr'] = interaction_counts['click'] / (interaction_counts['view'] + 1)

        # Конверсия покупок
        if 'purchase' in interaction_counts.columns and 'view' in interaction_counts.columns:
            item_stats['item_purchase_conversion'] = interaction_counts['purchase'] / (interaction_counts['view'] + 1)

        # Общий success ratio (например, положительные взаимодействия / все)
        if 'purchase' in interaction_counts.columns or 'click' in interaction_counts.columns:
            positive_cols = [col for col in ['purchase', 'click', 'add_to_cart'] if col in interaction_counts.columns]
            if positive_cols:
                positive_interactions = interaction_counts[positive_cols].sum(axis=1)
                total_interactions = interaction_counts.sum(axis=1)
                item_stats['item_success_ratio'] = positive_interactions / (total_interactions + 1)

    # 3. Более сложные статистики по времени
    if 'timestamp' in interactions.columns:
        interactions['timestamp'] = pd.to_datetime(interactions['timestamp'])

        # Скорость набора популярности
        first_interaction = interactions.groupby('item_id')['timestamp'].min().reset_index(name='first_interaction_time')
        last_interaction = interactions.groupby('item_id')['timestamp'].max().reset_index(name='last_interaction_time')

        time_info = first_interaction.merge(last_interaction, on='item_id')
        time_info['item_lifetime_days'] = (time_info['last_interaction_time'] - time_info['first_interaction_time']).dt.days + 1

        item_stats = item_stats.merge(time_info[['item_id', 'item_lifetime_days']], on='item_id', how='left')

        # Популярность в день (интенсивность)
        """        item_stats = item_stats.merge(
                    total_popularity.rename(columns={'item_popularity_total': 'total_interactions'}),
                    on='item_id', how='left'
                )
                item_stats['item_popularity_per_day'] = np.where(
                    item_stats['item_lifetime_days'] > 0,
                    item_stats['total_interactions'] / item_stats['item_lifetime_days'],
                    0
                )
        """
    return item_stats

In [4]:
def create_item_metadata_features(items_meta_df):
    """
    Создает признаки из метаданных объектов.
    """
    items_meta = items_meta_df.copy()
    features = pd.DataFrame()
    features['item_id'] = items_meta['item_id']

    # 1. Категориальные признаки (просто берем как есть)
    categorical_cols = ['category', 'brand', 'author', 'genre', 'color',
                       'subcategory', 'manufacturer', 'country']

    for col in categorical_cols:
        if col in items_meta.columns:
            features[f'item_{col}'] = items_meta[col].fillna('unknown')

    # 2. Иерархические категории (разбиваем путь)
    if 'category_path' in items_meta.columns:
        # Пример: "Электроника -> Смартфоны -> Apple"
        paths = items_meta['category_path'].str.split(' -> ', expand=True)

        for i in range(min(3, paths.shape[1])):  # Берем максимум 3 уровня
            features[f'item_category_level_{i+1}'] = paths[i].fillna('unknown')

    # 3. Текстовые признаки
    if 'title' in items_meta.columns:
        features['item_title_length'] = items_meta['title'].fillna('').apply(len)
        features['item_title_word_count'] = items_meta['title'].fillna('').apply(lambda x: len(str(x).split()))

        # Ключевые слова в названии
        keywords = ['sale', 'new', 'limited', 'exclusive', 'bestseller', 'discount']
        for keyword in keywords:
            features[f'item_title_has_{keyword}'] = items_meta['title'].fillna('').str.lower().str.contains(keyword).astype(int)

    if 'description' in items_meta.columns:
        features['item_description_length'] = items_meta['description'].fillna('').apply(len)
        features['item_has_description'] = (items_meta['description'].notna() &
                                           (items_meta['description'].str.strip() != '')).astype(int)

    # 4. Числовые признаки
    numeric_cols = ['price', 'weight', 'size', 'length', 'width', 'height', 'volume']

    for col in numeric_cols:
        if col in items_meta.columns:
            features[f'item_{col}'] = pd.to_numeric(items_meta[col], errors='coerce')

            # Логарифмирование для skewed распределений
            if features[f'item_{col}'].min() > 0:
                features[f'item_{col}_log'] = np.log1p(features[f'item_{col}'])

    # 5. Ценовые признаки
    if 'price' in items_meta.columns:
        # Признаки скидки
        if 'original_price' in items_meta.columns:
            features['item_discount_amount'] = items_meta['original_price'] - items_meta['price']
            features['item_discount_percentage'] = np.where(
                items_meta['original_price'] > 0,
                (items_meta['original_price'] - items_meta['price']) / items_meta['original_price'] * 100,
                0
            )
            features['item_is_discounted'] = (features['item_discount_percentage'] > 0).astype(int)
        else:
            # Если нет оригинальной цены, можно использовать медианную по категории
            if 'category' in items_meta.columns:
                median_price_by_category = items_meta.groupby('category')['price'].transform('median')
                features['item_price_vs_category_median'] = items_meta['price'] / (median_price_by_category + 1)

    # 6. Цена за единицу (если есть вес или объем)
    if 'price' in items_meta.columns and 'weight' in items_meta.columns:
        features['item_price_per_kg'] = np.where(
            items_meta['weight'] > 0,
            items_meta['price'] / items_meta['weight'],
            np.nan
        )

    if 'price' in items_meta.columns and 'volume' in items_meta.columns:
        features['item_price_per_liter'] = np.where(
            items_meta['volume'] > 0,
            items_meta['price'] / items_meta['volume'],
            np.nan
        )

    # 7. Бинарные признаки из категориальных
    if 'brand' in items_meta.columns:
        # Флаг для топ-брендов
        top_brands = items_meta['brand'].value_counts().head(10).index
        features['item_is_top_brand'] = items_meta['brand'].isin(top_brands).astype(int)

    return features

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

def create_text_features(items_meta_df, text_column='title', n_components=20):
    """
    Создает TF-IDF эмбеддинги для текстовых полей.

    Returns:
        DataFrame с эмбеддингами и оригинальный vectorizer для трансформации новых данных
    """
    if text_column not in items_meta_df.columns:
        return pd.DataFrame({'item_id': items_meta_df['item_id']}), None

    # Подготовка текста
    texts = items_meta_df[text_column].fillna('').astype(str).tolist()

    # TF-IDF векторзация
    vectorizer = TfidfVectorizer(
        max_features=1000,
        min_df=2,
        max_df=0.8,
        stop_words='english',  # или 'russian'
        ngram_range=(1, 2)
    )

    tfidf_matrix = vectorizer.fit_transform(texts)

    # Уменьшение размерности
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    embeddings = svd.fit_transform(tfidf_matrix)

    # Создаем DataFrame с эмбеддингами
    embedding_cols = [f'item_{text_column}_embed_{i}' for i in range(n_components)]
    embeddings_df = pd.DataFrame(embeddings, columns=embedding_cols)
    embeddings_df['item_id'] = items_meta_df['item_id'].values

    return embeddings_df, vectorizer

In [6]:
def create_all_item_features(interactions_df, items_meta_df, current_date=None):
    """
    Создает все признаки объектов в одном датафрейме.

    Args:
        interactions_df: DataFrame взаимодействий
        items_meta_df: DataFrame метаданных
        current_date: Дата для валидации/теста

    Returns:
        DataFrame со всеми признаками
    """
    print("Создание признаков популярности...")
    popularity_features = create_item_popularity_features(
        interactions_df, items_meta_df, current_date
    )

    print("Создание статистик взаимодействий...")
    interaction_features = create_item_interaction_stats(interactions_df)

    print("Создание признаков из метаданных...")
    metadata_features = create_item_metadata_features(items_meta_df)

    # Объединяем все признаки
    print("Объединение всех признаков...")
    all_features = popularity_features.copy()

    # Последовательно объединяем
    for features_df in [interaction_features, metadata_features]:
        all_features = all_features.merge(
            features_df,
            on='item_id',
            how='left'
        )

    # Дополнительно: нормализуем числовые признаки
    print("Нормализация числовых признаков...")
    numeric_cols = all_features.select_dtypes(include=[np.number]).columns.tolist()

    # Исключаем ID и бинарные признаки
    exclude_cols = ['item_id', 'is_new_item', 'item_is_discounted', 'item_is_top_brand']
    numeric_cols = [col for col in numeric_cols if col not in exclude_cols]

    # Min-Max нормализация
    for col in numeric_cols:
        if all_features[col].nunique() > 1:  # Не нормализуем константы
            min_val = all_features[col].min()
            max_val = all_features[col].max()
            if max_val > min_val:  # Избегаем деления на 0
                all_features[f'{col}_norm'] = (all_features[col] - min_val) / (max_val - min_val)

    print(f"Создано признаков: {all_features.shape[1] - 1}")  # минус item_id
    print(f"Количество объектов: {all_features.shape[0]}")

    return all_features

In [9]:
# Пример синтетических данных
np.random.seed(42)

# Создаем данные взаимодействий
n_interactions = 10000
n_items = 1000
n_users = 500

interactions_data = {
    'user_id': np.random.randint(1, n_users+1, n_interactions),
    'item_id': np.random.randint(1, n_items+1, n_interactions),
    'interaction_type': np.random.choice(['click', 'view', 'purchase', 'add_to_cart'],
                                        n_interactions, p=[0.5, 0.3, 0.15, 0.05]),
    'rating': np.random.choice([1, 2, 3, 4, 5, np.nan], n_interactions, p=[0.05, 0.1, 0.15, 0.3, 0.3, 0.1]),
    'timestamp': pd.date_range(start='2024-01-01', periods=n_interactions, freq='H')
}

interactions_df = pd.DataFrame(interactions_data)

# Создаем метаданные объектов
items_data = {
    'item_id': range(1, n_items+1),
    'category': np.random.choice(['Electronics', 'Books', 'Clothing', 'Home', 'Sports'], n_items),
    'brand': np.random.choice(['Brand_A', 'Brand_B', 'Brand_C', 'Brand_D', 'Brand_E', 'Unknown'], n_items),
    'price': np.random.exponential(100, n_items).round(2),
    'created_at': pd.date_range(start='2023-06-01', periods=n_items, freq='D'),
    'title': [f'Product_{i}' for i in range(1, n_items+1)],
    'description': [f'Description for product {i}' for i in range(1, n_items+1)]
}

items_meta_df = pd.DataFrame(items_data)

# Создаем все признаки
print("=" * 50)
print("СОЗДАНИЕ ПРИЗНАКОВ ДЛЯ ОБЪЕКТОВ")
print("=" * 50)

item_features_df = create_all_item_features(
    interactions_df=interactions_df,
    items_meta_df=items_meta_df,
    current_date='2024-02-01'
)

# Посмотрим на результат
print("\nПервые 5 строк финального датафрейма:")
print(item_features_df.head())

print("\nКолонки в финальном датафрейме:")
print(item_features_df.columns.tolist())

print("\nБазовая статистика по признакам:")
print(item_features_df.describe())

СОЗДАНИЕ ПРИЗНАКОВ ДЛЯ ОБЪЕКТОВ
Создание признаков популярности...
Создание статистик взаимодействий...
Создание признаков из метаданных...
Объединение всех признаков...
Нормализация числовых признаков...
Создано признаков: 62
Количество объектов: 1000

Первые 5 строк финального датафрейма:
   item_id  item_popularity_total  item_popularity_30d  item_popularity_7d  \
0      182                      8                    7                   6   
1       13                     14                   13                  13   
2      339                      8                    7                   7   
3      797                      7                    6                   6   
4      259                     14                   13                  13   

   item_popularity_1d  item_popularity_trend  item_popularity_std  \
0                   6               1.000000             0.179605   
1                  12               1.818182             0.179605   
2                   7           

In [10]:
def create_user_activity_features(interactions_df, users_meta_df=None, current_date=None):
    """
    Создает признаки активности пользователей.

    Args:
        interactions_df: DataFrame с колонками ['user_id', 'timestamp', 'interaction_type']
        users_meta_df: DataFrame с метаданными пользователей (опционально)
        current_date: Дата для расчета временных окон

    Returns:
        DataFrame с признаками пользователей
    """
    if current_date is None:
        current_date = interactions_df['timestamp'].max()
    else:
        current_date = pd.to_datetime(current_date)

    interactions = interactions_df.copy()
    interactions['timestamp'] = pd.to_datetime(interactions['timestamp'])

    # Создаем базовый DataFrame
    user_features = pd.DataFrame()
    user_features['user_id'] = interactions['user_id'].unique()

    # 1. Общая активность
    user_activity = interactions.groupby('user_id').size().reset_index(name='user_activity_count')
    user_features = user_features.merge(user_activity, on='user_id', how='left')
    user_features['user_activity_count'] = user_features['user_activity_count'].fillna(0).astype(int)

    # 2. Активность по временным окнам
    one_day_ago = current_date - timedelta(days=1)
    seven_days_ago = current_date - timedelta(days=7)
    thirty_days_ago = current_date - timedelta(days=30)

    # Последние 30 дней
    recent_30d = interactions[interactions['timestamp'] >= thirty_days_ago]
    activity_30d = recent_30d.groupby('user_id').size().reset_index(name='user_activity_30d')
    user_features = user_features.merge(activity_30d, on='user_id', how='left')
    user_features['user_activity_30d'] = user_features['user_activity_30d'].fillna(0).astype(int)

    # Последние 7 дней
    recent_7d = interactions[interactions['timestamp'] >= seven_days_ago]
    activity_7d = recent_7d.groupby('user_id').size().reset_index(name='user_activity_7d')
    user_features = user_features.merge(activity_7d, on='user_id', how='left')
    user_features['user_activity_7d'] = user_features['user_activity_7d'].fillna(0).astype(int)

    # Последние 1 день
    recent_1d = interactions[interactions['timestamp'] >= one_day_ago]
    activity_1d = recent_1d.groupby('user_id').size().reset_index(name='user_activity_1d')
    user_features = user_features.merge(activity_1d, on='user_id', how='left')
    user_features['user_activity_1d'] = user_features['user_activity_1d'].fillna(0).astype(int)

    # 3. Частота активности (интервалы между действиями)
    # Даты первого и последнего взаимодействия
    user_time_stats = interactions.groupby('user_id')['timestamp'].agg(['min', 'max']).reset_index()
    user_time_stats.columns = ['user_id', 'first_interaction', 'last_interaction']

    user_time_stats['user_age_in_system_days'] = (current_date - user_time_stats['first_interaction']).dt.days + 1
    user_time_stats['user_recency_days'] = (current_date - user_time_stats['last_interaction']).dt.days

    user_features = user_features.merge(
        user_time_stats[['user_id', 'user_age_in_system_days', 'user_recency_days']],
        on='user_id', how='left'
    )

    # Частота активности
    user_features['user_activity_frequency'] = np.where(
        user_features['user_age_in_system_days'] > 0,
        user_features['user_activity_count'] / user_features['user_age_in_system_days'],
        0
    )

    # 4. Сессионные признаки (если есть session_id)
    if 'session_id' in interactions.columns:
        session_stats = interactions.groupby(['user_id', 'session_id']).size().reset_index(name='session_length')
        user_session_stats = session_stats.groupby('user_id').agg({
            'session_id': 'count',
            'session_length': 'mean'
        }).reset_index()

        user_session_stats.columns = ['user_id', 'user_session_count', 'avg_session_length']

        user_features = user_features.merge(user_session_stats, on='user_id', how='left')
        user_features['user_session_count'] = user_features['user_session_count'].fillna(0).astype(int)
        user_features['avg_session_length'] = user_features['avg_session_length'].fillna(0)
    else:
        # Приближенный расчет сессий по времени
        interactions_sorted = interactions.sort_values(['user_id', 'timestamp'])
        interactions_sorted['time_diff'] = interactions_sorted.groupby('user_id')['timestamp'].diff()

        # Сессия заканчивается, если перерыв больше 30 минут
        interactions_sorted['new_session'] = (interactions_sorted['time_diff'] > timedelta(minutes=30)) | \
                                            (interactions_sorted['time_diff'].isna())
        interactions_sorted['session_id'] = interactions_sorted.groupby('user_id')['new_session'].cumsum()

        session_stats = interactions_sorted.groupby(['user_id', 'session_id']).size().reset_index(name='session_length')
        user_session_stats = session_stats.groupby('user_id').agg({
            'session_id': 'count',
            'session_length': 'mean'
        }).reset_index()

        user_session_stats.columns = ['user_id', 'user_session_count', 'avg_session_length']

        user_features = user_features.merge(user_session_stats, on='user_id', how='left')
        user_features['user_session_count'] = user_features['user_session_count'].fillna(0).astype(int)
        user_features['avg_session_length'] = user_features['avg_session_length'].fillna(0)

    return user_features

In [11]:
def create_user_behavioral_features(interactions_df, items_meta_df):
    """
    Создает поведенческие признаки пользователей.

    Args:
        interactions_df: DataFrame с колонками ['user_id', 'item_id', 'rating', 'timestamp']
        items_meta_df: DataFrame с метаданными объектов
    """
    interactions = interactions_df.copy()
    items_meta = items_meta_df.copy()

    # Объединяем взаимодействия с метаданными объектов
    interactions_with_meta = interactions.merge(
        items_meta[['item_id', 'category', 'price']],
        on='item_id',
        how='left'
    )

    user_features = pd.DataFrame()
    user_features['user_id'] = interactions['user_id'].unique()

    # 1. Средний рейтинг пользователя
    if 'rating' in interactions.columns:
        user_avg_rating = interactions[interactions['rating'].notna()].groupby('user_id')['rating'].mean().reset_index(name='user_avg_rating')
        user_features = user_features.merge(user_avg_rating, on='user_id', how='left')
        user_features['user_avg_rating'] = user_features['user_avg_rating'].fillna(0)

    # 2. Предпочитаемая категория
    if 'category' in interactions_with_meta.columns:
        # Самая частая категория
        user_category_counts = interactions_with_meta.groupby(['user_id', 'category']).size().reset_index(name='category_count')
        user_preferred_category = user_category_counts.loc[
            user_category_counts.groupby('user_id')['category_count'].idxmax()
        ][['user_id', 'category']].rename(columns={'category': 'user_preferred_category'})

        user_features = user_features.merge(user_preferred_category, on='user_id', how='left')

        # Диверсификация (энтропия категорий)
        user_category_distribution = user_category_counts.pivot_table(
            index='user_id',
            columns='category',
            values='category_count',
            fill_value=0
        )

        # Нормализуем распределение
        user_category_distribution_norm = user_category_distribution.div(
            user_category_distribution.sum(axis=1), axis=0
        )

        # Считаем энтропию
        user_entropy = pd.Series(
            entropy(user_category_distribution_norm.values.T, base=2),
            index=user_category_distribution_norm.index
        ).reset_index(name='user_diversity')

        user_features = user_features.merge(user_entropy, on='user_id', how='left')
        user_features['user_diversity'] = user_features['user_diversity'].fillna(0)

    # 3. Предпочитаемый ценовой сегмент
    if 'price' in interactions_with_meta.columns:
        user_price_stats = interactions_with_meta[interactions_with_meta['price'].notna()].groupby('user_id')['price'].agg([
            'median', 'mean', 'std', 'min', 'max'
        ]).reset_index()

        user_price_stats.columns = ['user_id', 'user_preferred_price_median',
                                   'user_avg_price', 'user_price_std',
                                   'user_min_price', 'user_max_price']

        user_features = user_features.merge(user_price_stats, on='user_id', how='left')

        # Размах цен (широта интересов)
        user_features['user_price_range'] = user_features['user_max_price'] - user_features['user_min_price']
        user_features['user_price_variability'] = user_features['user_price_std'] / (user_features['user_avg_price'] + 1)

    # 4. Предпочитаемые бренды (если есть в метаданных)
    if 'brand' in items_meta.columns:
        interactions_with_brand = interactions.merge(
            items_meta[['item_id', 'brand']],
            on='item_id',
            how='left'
        )

        user_brand_counts = interactions_with_brand.groupby(['user_id', 'brand']).size().reset_index(name='brand_count')
        user_preferred_brand = user_brand_counts.loc[
            user_brand_counts.groupby('user_id')['brand_count'].idxmax()
        ][['user_id', 'brand']].rename(columns={'brand': 'user_preferred_brand'})

        user_features = user_features.merge(user_preferred_brand, on='user_id', how='left')

    # 5. Количество уникальных категорий/брендов
    if 'category' in interactions_with_meta.columns:
        user_unique_categories = interactions_with_meta.groupby('user_id')['category'].nunique().reset_index(name='user_unique_categories_count')
        user_features = user_features.merge(user_unique_categories, on='user_id', how='left')

    return user_features

In [12]:
def create_user_temporal_patterns(interactions_df):
    """
    Создает временные паттерны пользователей.
    """
    interactions = interactions_df.copy()
    interactions['timestamp'] = pd.to_datetime(interactions['timestamp'])

    user_features = pd.DataFrame()
    user_features['user_id'] = interactions['user_id'].unique()

    # 1. Время суток
    interactions['hour'] = interactions['timestamp'].dt.hour

    # Разбиваем на части дня
    def get_time_of_day(hour):
        if 5 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 17:
            return 'afternoon'
        elif 17 <= hour < 22:
            return 'evening'
        else:
            return 'night'

    interactions['time_of_day'] = interactions['hour'].apply(get_time_of_day)

    # Самое частое время активности
    time_of_day_counts = interactions.groupby(['user_id', 'time_of_day']).size().reset_index(name='count')
    user_primary_time = time_of_day_counts.loc[
        time_of_day_counts.groupby('user_id')['count'].idxmax()
    ][['user_id', 'time_of_day']].rename(columns={'time_of_day': 'user_primary_time_of_day'})

    user_features = user_features.merge(user_primary_time, on='user_id', how='left')

    # Распределение по времени суток
    time_of_day_encoded = pd.get_dummies(interactions['time_of_day'], prefix='user_time_of_day')
    time_of_day_encoded['user_id'] = interactions['user_id'].values

    time_distribution = time_of_day_encoded.groupby('user_id').mean().reset_index()
    user_features = user_features.merge(time_distribution, on='user_id', how='left')

    # 2. Дни недели
    interactions['day_of_week'] = interactions['timestamp'].dt.dayofweek
    interactions['is_weekend'] = interactions['day_of_week'].isin([5, 6]).astype(int)

    # Процент активности на выходных
    weekend_stats = interactions.groupby('user_id')['is_weekend'].mean().reset_index(name='user_weekend_activity_ratio')
    user_features = user_features.merge(weekend_stats, on='user_id', how='left')

    # Флаг "пользователь выходного дня"
    user_features['user_is_weekend_user'] = (user_features['user_weekend_activity_ratio'] > 0.5).astype(int)

    # Распределение по дням недели
    day_of_week_encoded = pd.get_dummies(interactions['day_of_week'], prefix='user_day_of_week')
    day_of_week_encoded['user_id'] = interactions['user_id'].values

    day_distribution = day_of_week_encoded.groupby('user_id').mean().reset_index()
    user_features = user_features.merge(day_distribution, on='user_id', how='left')

    # 3. Временные интервалы между действиями
    interactions_sorted = interactions.sort_values(['user_id', 'timestamp'])
    interactions_sorted['time_diff'] = interactions_sorted.groupby('user_id')['timestamp'].diff()

    time_diff_stats = interactions_sorted[interactions_sorted['time_diff'].notna()].groupby('user_id')['time_diff'].agg([
        'mean', 'median', 'std'
    ]).reset_index()

    time_diff_stats.columns = ['user_id', 'user_avg_time_between_actions',
                              'user_median_time_between_actions', 'user_time_between_actions_std']

    # Конвертируем в секунды
    for col in ['user_avg_time_between_actions', 'user_median_time_between_actions',
                'user_time_between_actions_std']:
        time_diff_stats[col] = time_diff_stats[col].dt.total_seconds()

    user_features = user_features.merge(time_diff_stats, on='user_id', how='left')

    return user_features

In [13]:
def create_user_demographic_features(users_meta_df, current_date=None):
    """
    Создает демографические признаки пользователей.
    """
    if users_meta_df is None:
        return pd.DataFrame()

    users_meta = users_meta_df.copy()
    user_features = users_meta[['user_id']].copy()

    if current_date is None:
        current_date = datetime.now()

    # 1. Возраст пользователя (если есть дата рождения)
    if 'birth_date' in users_meta.columns:
        users_meta['birth_date'] = pd.to_datetime(users_meta['birth_date'])
        user_features['user_age'] = ((current_date - users_meta['birth_date']).dt.days / 365.25).astype(int)

        # Возрастные группы
        bins = [0, 18, 25, 35, 45, 55, 65, 100]
        labels = ['<18', '18-25', '26-35', '36-45', '46-55', '56-65', '65+']
        user_features['user_age_group'] = pd.cut(user_features['user_age'], bins=bins, labels=labels, right=False)

    # 2. Пол
    if 'gender' in users_meta.columns:
        user_features['user_gender'] = users_meta['gender'].fillna('unknown')
        # One-hot encoding
        gender_dummies = pd.get_dummies(user_features['user_gender'], prefix='user_gender')
        user_features = pd.concat([user_features, gender_dummies], axis=1)

    # 3. Локация
    if 'city' in users_meta.columns:
        user_features['user_city'] = users_meta['city'].fillna('unknown')

    if 'country' in users_meta.columns:
        user_features['user_country'] = users_meta['country'].fillna('unknown')

        # Топ стран
        top_countries = users_meta['country'].value_counts().head(20).index
        user_features['user_is_top_country'] = user_features['user_country'].isin(top_countries).astype(int)

    # 4. Дата регистрации
    if 'registration_date' in users_meta.columns:
        users_meta['registration_date'] = pd.to_datetime(users_meta['registration_date'])
        user_features['user_days_since_registration'] = (current_date - users_meta['registration_date']).dt.days

        # Группы по давности регистрации
        reg_bins = [0, 30, 90, 180, 365, 730, 1825, 10000]
        reg_labels = ['<1m', '1-3m', '3-6m', '6-12m', '1-2y', '2-5y', '5y+']
        user_features['user_registration_group'] = pd.cut(
            user_features['user_days_since_registration'],
            bins=reg_bins,
            labels=reg_labels,
            right=False
        )

    # 5. Доход (если есть)
    if 'income' in users_meta.columns:
        user_features['user_income'] = users_meta['income']

        # Сегменты дохода
        income_bins = [0, 30000, 60000, 100000, 150000, 300000, float('inf')]
        income_labels = ['<30k', '30-60k', '60-100k', '100-150k', '150-300k', '300k+']
        user_features['user_income_segment'] = pd.cut(
            user_features['user_income'],
            bins=income_bins,
            labels=income_labels,
            right=False
        )

    # 6. Образование, профессия и т.д.
    categorical_cols = ['education', 'occupation', 'marital_status', 'device_type']

    for col in categorical_cols:
        if col in users_meta.columns:
            user_features[f'user_{col}'] = users_meta[col].fillna('unknown')

    return user_features

In [14]:
def create_user_item_interaction_features(interactions_df, current_date=None):
    """
    Создает признаки взаимодействия пользователь-объект.
    """
    if current_date is None:
        current_date = interactions_df['timestamp'].max()
    else:
        current_date = pd.to_datetime(current_date)

    interactions = interactions_df.copy()
    interactions['timestamp'] = pd.to_datetime(interactions['timestamp'])

    # Создаем базовый DataFrame со всеми парами user-item
    user_item_pairs = interactions[['user_id', 'item_id']].drop_duplicates()

    # 1. Количество взаимодействий с конкретным объектом
    interaction_counts = interactions.groupby(['user_id', 'item_id']).size().reset_index(name='user_item_interaction_count')
    user_item_features = user_item_pairs.merge(interaction_counts, on=['user_id', 'item_id'], how='left')
    user_item_features['user_item_interaction_count'] = user_item_features['user_item_interaction_count'].fillna(0).astype(int)

    # 2. Временные признаки взаимодействия
    time_stats = interactions.groupby(['user_id', 'item_id'])['timestamp'].agg([
        'min', 'max', 'count'
    ]).reset_index()

    time_stats.columns = ['user_id', 'item_id', 'user_item_first_interaction',
                         'user_item_last_interaction', 'interaction_count_check']

    user_item_features = user_item_features.merge(
        time_stats[['user_id', 'item_id', 'user_item_first_interaction', 'user_item_last_interaction']],
        on=['user_id', 'item_id'], how='left'
    )

    # Дни с первого и последнего взаимодействия
    user_item_features['user_item_first_interaction_days_ago'] = (
        current_date - user_item_features['user_item_first_interaction']
    ).dt.days.fillna(9999).astype(int)

    user_item_features['user_item_last_interaction_days_ago'] = (
        current_date - user_item_features['user_item_last_interaction']
    ).dt.days.fillna(9999).astype(int)

    # 3. Средний рейтинг пользователя объекту
    if 'rating' in interactions.columns:
        avg_rating = interactions[interactions['rating'].notna()].groupby(['user_id', 'item_id'])['rating'].mean().reset_index(name='user_item_avg_rating')
        user_item_features = user_item_features.merge(avg_rating, on=['user_id', 'item_id'], how='left')
        user_item_features['user_item_avg_rating'] = user_item_features['user_item_avg_rating'].fillna(0)

    # 4. Типы взаимодействий (если есть interaction_type)
    if 'interaction_type' in interactions.columns:
        # Количество взаимодействий по типам
        interaction_type_counts = interactions.groupby(['user_id', 'item_id', 'interaction_type']).size().unstack(fill_value=0).reset_index()

        # Переименовываем колонки
        interaction_type_counts.columns = [f'user_item_{col}_count' if col != 'user_id' and col != 'item_id' else col
                                          for col in interaction_type_counts.columns]

        user_item_features = user_item_features.merge(interaction_type_counts, on=['user_id', 'item_id'], how='left')

        # Заполняем пропуски нулями
        for col in interaction_type_counts.columns:
            if col not in ['user_id', 'item_id']:
                user_item_features[col] = user_item_features[col].fillna(0).astype(int)

    # 5. Интенсивность взаимодействий (взаимодействий в день)
    user_item_features['user_item_interaction_intensity'] = np.where(
        user_item_features['user_item_first_interaction_days_ago'] < 9999,
        user_item_features['user_item_interaction_count'] /
        (user_item_features['user_item_first_interaction_days_ago'] + 1),
        0
    )

    # 6. Флаги повторного взаимодействия
    user_item_features['user_item_has_repeated_interaction'] = (user_item_features['user_item_interaction_count'] > 1).astype(int)
    user_item_features['user_item_is_first_interaction_recent'] = (user_item_features['user_item_first_interaction_days_ago'] <= 7).astype(int)

    return user_item_features

In [15]:
def create_user_item_matching_features(user_features, item_features, interactions_df, items_meta_df):
    """
    Создает признаки совпадения профилей пользователя и объекта.
    """
    # Базовый DataFrame с парами user-item
    user_item_pairs = interactions_df[['user_id', 'item_id']].drop_duplicates()

    matching_features = user_item_pairs.copy()

    # 1. Совпадение категории
    if 'user_preferred_category' in user_features.columns and 'item_category' in item_features.columns:
        # Получаем категории пользователей и объектов
        user_categories = user_features[['user_id', 'user_preferred_category']]
        item_categories = item_features[['item_id', 'item_category']]

        # Объединяем
        matching_data = matching_features.merge(user_categories, on='user_id', how='left')
        matching_data = matching_data.merge(item_categories, on='item_id', how='left')

        matching_data['category_match'] = (matching_data['user_preferred_category'] == matching_data['item_category']).astype(int)
        matching_data['preferred_category_match'] = matching_data['category_match']  # Для данного примера

        matching_features = matching_features.merge(
            matching_data[['user_id', 'item_id', 'category_match', 'preferred_category_match']],
            on=['user_id', 'item_id'], how='left'
        )

    # 2. Совпадение ценового сегмента
    if 'user_preferred_price_median' in user_features.columns and 'item_price' in item_features.columns:
        user_prices = user_features[['user_id', 'user_preferred_price_median', 'user_price_std']]
        item_prices = item_features[['item_id', 'item_price']]

        price_data = matching_features.merge(user_prices, on='user_id', how='left')
        price_data = price_data.merge(item_prices, on='item_id', how='left')

        # Разница между ценой объекта и предпочитаемой ценой пользователя
        price_data['price_diff'] = price_data['item_price'] - price_data['user_preferred_price_median']
        price_data['price_diff_abs'] = price_data['price_diff'].abs()

        # Нормализованная разница (в стандартных отклонениях)
        price_data['price_diff_norm'] = np.where(
            price_data['user_price_std'] > 0,
            price_data['price_diff_abs'] / price_data['user_price_std'],
            0
        )

        # Бинарный признак совпадения (в пределах 1.5 стандартных отклонений)
        price_data['price_match'] = (price_data['price_diff_norm'] <= 1.5).astype(int)

        matching_features = matching_features.merge(
            price_data[['user_id', 'item_id', 'price_diff', 'price_diff_abs', 'price_diff_norm', 'price_match']],
            on=['user_id', 'item_id'], how='left'
        )

    # 3. Совпадение бренда
    if 'user_preferred_brand' in user_features.columns and 'item_brand' in item_features.columns:
        user_brands = user_features[['user_id', 'user_preferred_brand']]
        item_brands = item_features[['item_id', 'item_brand']]

        brand_data = matching_features.merge(user_brands, on='user_id', how='left')
        brand_data = brand_data.merge(item_brands, on='item_id', how='left')

        brand_data['brand_match'] = (brand_data['user_preferred_brand'] == brand_data['item_brand']).astype(int)

        matching_features = matching_features.merge(
            brand_data[['user_id', 'item_id', 'brand_match']],
            on=['user_id', 'item_id'], how='left'
        )

    # 4. Историческое взаимодействие с категорией/брендом
    interactions_with_meta = interactions_df.merge(
        items_meta_df[['item_id', 'category', 'brand']],
        on='item_id',
        how='left'
    )

    # Взаимодействие с категорией
    if 'category' in interactions_with_meta.columns:
        user_category_interaction = interactions_with_meta.groupby(['user_id', 'category']).size().reset_index(name='category_interaction_count')
        user_category_interaction['user_has_category_interaction'] = 1

        # Добавляем к matching_features
        matching_data_with_cat = matching_features.merge(
            item_features[['item_id', 'item_category']],
            on='item_id',
            how='left'
        )

        matching_data_with_cat = matching_data_with_cat.merge(
            user_category_interaction[['user_id', 'category', 'user_has_category_interaction', 'category_interaction_count']],
            left_on=['user_id', 'item_category'],
            right_on=['user_id', 'category'],
            how='left'
        )

        matching_features['user_has_category_interaction'] = matching_data_with_cat['user_has_category_interaction'].fillna(0).astype(int)
        matching_features['user_category_interaction_count'] = matching_data_with_cat['category_interaction_count'].fillna(0)

    return matching_features

In [16]:
def create_user_item_matching_features(user_features, item_features, interactions_df, items_meta_df):
    """
    Создает признаки совпадения профилей пользователя и объекта.
    """
    # Базовый DataFrame с парами user-item
    user_item_pairs = interactions_df[['user_id', 'item_id']].drop_duplicates()

    matching_features = user_item_pairs.copy()

    # 1. Совпадение категории
    if 'user_preferred_category' in user_features.columns and 'item_category' in item_features.columns:
        # Получаем категории пользователей и объектов
        user_categories = user_features[['user_id', 'user_preferred_category']]
        item_categories = item_features[['item_id', 'item_category']]

        # Объединяем
        matching_data = matching_features.merge(user_categories, on='user_id', how='left')
        matching_data = matching_data.merge(item_categories, on='item_id', how='left')

        matching_data['category_match'] = (matching_data['user_preferred_category'] == matching_data['item_category']).astype(int)
        matching_data['preferred_category_match'] = matching_data['category_match']  # Для данного примера

        matching_features = matching_features.merge(
            matching_data[['user_id', 'item_id', 'category_match', 'preferred_category_match']],
            on=['user_id', 'item_id'], how='left'
        )

    # 2. Совпадение ценового сегмента
    if 'user_preferred_price_median' in user_features.columns and 'item_price' in item_features.columns:
        user_prices = user_features[['user_id', 'user_preferred_price_median', 'user_price_std']]
        item_prices = item_features[['item_id', 'item_price']]

        price_data = matching_features.merge(user_prices, on='user_id', how='left')
        price_data = price_data.merge(item_prices, on='item_id', how='left')

        # Разница между ценой объекта и предпочитаемой ценой пользователя
        price_data['price_diff'] = price_data['item_price'] - price_data['user_preferred_price_median']
        price_data['price_diff_abs'] = price_data['price_diff'].abs()

        # Нормализованная разница (в стандартных отклонениях)
        price_data['price_diff_norm'] = np.where(
            price_data['user_price_std'] > 0,
            price_data['price_diff_abs'] / price_data['user_price_std'],
            0
        )

        # Бинарный признак совпадения (в пределах 1.5 стандартных отклонений)
        price_data['price_match'] = (price_data['price_diff_norm'] <= 1.5).astype(int)

        matching_features = matching_features.merge(
            price_data[['user_id', 'item_id', 'price_diff', 'price_diff_abs', 'price_diff_norm', 'price_match']],
            on=['user_id', 'item_id'], how='left'
        )

    # 3. Совпадение бренда
    if 'user_preferred_brand' in user_features.columns and 'item_brand' in item_features.columns:
        user_brands = user_features[['user_id', 'user_preferred_brand']]
        item_brands = item_features[['item_id', 'item_brand']]

        brand_data = matching_features.merge(user_brands, on='user_id', how='left')
        brand_data = brand_data.merge(item_brands, on='item_id', how='left')

        brand_data['brand_match'] = (brand_data['user_preferred_brand'] == brand_data['item_brand']).astype(int)

        matching_features = matching_features.merge(
            brand_data[['user_id', 'item_id', 'brand_match']],
            on=['user_id', 'item_id'], how='left'
        )

    # 4. Историческое взаимодействие с категорией/брендом
    interactions_with_meta = interactions_df.merge(
        items_meta_df[['item_id', 'category', 'brand']],
        on='item_id',
        how='left'
    )

    # Взаимодействие с категорией
    if 'category' in interactions_with_meta.columns:
        user_category_interaction = interactions_with_meta.groupby(['user_id', 'category']).size().reset_index(name='category_interaction_count')
        user_category_interaction['user_has_category_interaction'] = 1

        # Добавляем к matching_features
        matching_data_with_cat = matching_features.merge(
            item_features[['item_id', 'item_category']],
            on='item_id',
            how='left'
        )

        matching_data_with_cat = matching_data_with_cat.merge(
            user_category_interaction[['user_id', 'category', 'user_has_category_interaction', 'category_interaction_count']],
            left_on=['user_id', 'item_category'],
            right_on=['user_id', 'category'],
            how='left'
        )

        matching_features['user_has_category_interaction'] = matching_data_with_cat['user_has_category_interaction'].fillna(0).astype(int)
        matching_features['user_category_interaction_count'] = matching_data_with_cat['category_interaction_count'].fillna(0)

    return matching_features

In [17]:
def create_similarity_based_features(user_features, item_features, interactions_df,
                                   user_similarity_matrix=None, item_similarity_matrix=None,
                                   n_similar=10):
    """
    Создает признаки на основе похожих объектов и пользователей.
    """
    user_item_pairs = interactions_df[['user_id', 'item_id']].drop_duplicates()
    similarity_features = user_item_pairs.copy()

    # 1. Признаки на основе похожих объектов
    if item_similarity_matrix is not None and 'user_item_interaction_count' in interactions_df.columns:
        print("Создание признаков на основе похожих объектов...")

        # Получаем историю взаимодействий пользователей
        user_history = interactions_df.groupby(['user_id', 'item_id'])['user_item_interaction_count'].sum().reset_index()

        # Для каждой пары user-item считаем статистики по похожим объектам
        similar_items_stats = []

        for idx, row in user_item_pairs.iterrows():
            user_id = row['user_id']
            item_id = row['item_id']

            # Находим N самых похожих объектов
            if item_id in item_similarity_matrix.index:
                similar_items = item_similarity_matrix.loc[item_id].nlargest(n_similar + 1)[1:]  # Исключаем сам объект
                similar_item_ids = similar_items.index.tolist()

                # История пользователя с похожими объектами
                user_similar_history = user_history[
                    (user_history['user_id'] == user_id) &
                    (user_history['item_id'].isin(similar_item_ids))
                ]

                # Агрегируем статистики
                if not user_similar_history.empty:
                    avg_rating_similar = user_similar_history['user_item_interaction_count'].mean()
                    total_interactions_similar = user_similar_history['user_item_interaction_count'].sum()
                else:
                    avg_rating_similar = 0
                    total_interactions_similar = 0

                similar_items_stats.append({
                    'user_id': user_id,
                    'item_id': item_id,
                    'user_avg_rating_on_similar_items': avg_rating_similar,
                    'user_interaction_count_with_similar_items': total_interactions_similar
                })

            if idx % 1000 == 0 and idx > 0:
                print(f"Обработано {idx} пар...")

        similar_stats_df = pd.DataFrame(similar_items_stats)
        similarity_features = similarity_features.merge(similar_stats_df, on=['user_id', 'item_id'], how='left')

    # 2. Признаки на основе похожих пользователей
    if user_similarity_matrix is not None:
        print("Создание признаков на основе похожих пользователей...")

        # Получаем популярность объектов среди пользователей
        item_popularity = interactions_df.groupby('item_id')['user_item_interaction_count'].sum().reset_index(name='item_popularity_total')

        similar_users_stats = []

        for idx, row in user_item_pairs.iterrows():
            user_id = row['user_id']
            item_id = row['item_id']

            # Находим K самых похожих пользователей
            if user_id in user_similarity_matrix.index:
                similar_users = user_similarity_matrix.loc[user_id].nlargest(n_similar + 1)[1:]  # Исключаем самого пользователя
                similar_user_ids = similar_users.index.tolist()

                # Взаимодействия похожих пользователей с данным объектом
                similar_users_interactions = interactions_df[
                    (interactions_df['user_id'].isin(similar_user_ids)) &
                    (interactions_df['item_id'] == item_id)
                ]

                # Агрегируем статистики
                if not similar_users_interactions.empty:
                    avg_popularity_similar = similar_users_interactions['user_item_interaction_count'].mean()
                    avg_rating_similar = similar_users_interactions['rating'].mean() if 'rating' in similar_users_interactions.columns else 0
                else:
                    avg_popularity_similar = 0
                    avg_rating_similar = 0

                similar_users_stats.append({
                    'user_id': user_id,
                    'item_id': item_id,
                    'avg_popularity_of_item_among_similar_users': avg_popularity_similar,
                    'avg_rating_of_item_among_similar_users': avg_rating_similar
                })

            if idx % 1000 == 0 and idx > 0:
                print(f"Обработано {idx} пар...")

        similar_users_df = pd.DataFrame(similar_users_stats)
        similarity_features = similarity_features.merge(similar_users_df, on=['user_id', 'item_id'], how='left')

    return similarity_features

In [18]:
def create_rank_features(user_item_pairs, user_features, item_features, interactions_df, items_meta_df):
    """
    Создает ранговые признаки для пар пользователь-объект.
    """
    rank_features = user_item_pairs.copy()

    # 1. Ранг популярности объекта в категории пользователя
    if 'user_preferred_category' in user_features.columns and 'item_category' in item_features.columns:
        # Получаем категорию объекта и популярность
        item_popularity = interactions_df.groupby('item_id').size().reset_index(name='item_popularity')

        # Добавляем категории объектов
        item_popularity_with_cat = item_popularity.merge(
            item_features[['item_id', 'item_category']],
            on='item_id',
            how='left'
        )

        # Считаем ранг популярности внутри каждой категории
        item_popularity_with_cat['item_popularity_rank_in_category'] = item_popularity_with_cat.groupby('item_category')['item_popularity'].rank(method='dense', ascending=False)

        # Для каждой пары user-item определяем ранг
        user_preferred_cats = user_features[['user_id', 'user_preferred_category']]

        rank_data = rank_features.merge(user_preferred_cats, on='user_id', how='left')
        rank_data = rank_data.merge(
            item_popularity_with_cat[['item_id', 'item_category', 'item_popularity_rank_in_category']],
            on='item_id', how='left'
        )

        # Ранг в предпочитаемой категории пользователя
        rank_data['item_popularity_rank_in_user_category'] = np.where(
            rank_data['user_preferred_category'] == rank_data['item_category'],
            rank_data['item_popularity_rank_in_category'],
            999  # Если категория не совпадает, ставим высокий ранг
        )

        rank_features = rank_features.merge(
            rank_data[['user_id', 'item_id', 'item_popularity_rank_in_user_category']],
            on=['user_id', 'item_id'], how='left'
        )

    # 2. Ранг активности пользователя
    if 'user_activity_rank' in user_features.columns:
        user_ranks = user_features[['user_id', 'user_activity_rank']]
        rank_features = rank_features.merge(user_ranks, on='user_id', how='left')

    # 3. Относительный ранг цены объекта для пользователя
    if 'user_preferred_price_median' in user_features.columns and 'item_price' in item_features.columns:
        user_prices = user_features[['user_id', 'user_preferred_price_median', 'user_min_price', 'user_max_price']]
        item_prices = item_features[['item_id', 'item_price']]

        price_rank_data = rank_features.merge(user_prices, on='user_id', how='left')
        price_rank_data = price_rank_data.merge(item_prices, on='item_id', how='left')

        # Ранг цены относительно предпочтений пользователя
        price_rank_data['price_rank_for_user'] = np.where(
            price_rank_data['item_price'] < price_rank_data['user_preferred_price_median'],
            'below_preferred',
            np.where(
                price_rank_data['item_price'] > price_rank_data['user_preferred_price_median'],
                'above_preferred',
                'at_preferred'
            )
        )

        # Кодируем
        price_rank_encoded = pd.get_dummies(price_rank_data['price_rank_for_user'], prefix='price_rank')
        price_rank_encoded[['user_id', 'item_id']] = price_rank_data[['user_id', 'item_id']].values

        rank_features = rank_features.merge(
            price_rank_encoded,
            on=['user_id', 'item_id'],
            how='left'
        )

    # 4. Квантили разницы во времени
    if 'user_item_last_interaction_days_ago' in rank_features.columns:
        rank_features['recency_quantile'] = pd.qcut(
            rank_features['user_item_last_interaction_days_ago'],
            q=5,
            labels=['very_old', 'old', 'medium', 'recent', 'very_recent']
        )

    return rank_features

In [19]:
def create_all_user_item_features(interactions_df, user_features, item_features,
                                items_meta_df, current_date=None,
                                user_similarity_matrix=None, item_similarity_matrix=None):
    """
    Создает все признаки для пар пользователь-объект.
    """
    print("=" * 50)
    print("СОЗДАНИЕ ПРИЗНАКОВ USER-ITEM ВЗАИМОДЕЙСТВИЙ")
    print("=" * 50)

    print("1. Создание базовых признаков взаимодействия...")
    interaction_features = create_user_item_interaction_features(interactions_df, current_date)

    print("2. Создание признаков совпадения профилей...")
    matching_features = create_user_item_matching_features(
        user_features, item_features, interactions_df, items_meta_df
    )

    print("3. Создание признаков на основе похожих объектов/пользователей...")
    similarity_features = create_similarity_based_features(
        user_features, item_features, interactions_df,
        user_similarity_matrix, item_similarity_matrix
    )

    print("4. Создание ранговых признаков...")
    rank_features = create_rank_features(
        interactions_df[['user_id', 'item_id']].drop_duplicates(),
        user_features, item_features, interactions_df, items_meta_df
    )

    print("5. Объединение всех признаков...")
    # Объединяем все признаки
    all_user_item_features = interaction_features.copy()

    for features_df in [matching_features, similarity_features, rank_features]:
        all_user_item_features = all_user_item_features.merge(
            features_df,
            on=['user_id', 'item_id'],
            how='left'
        )

    # Заполняем пропуски
    print("6. Заполнение пропусков...")
    numeric_cols = all_user_item_features.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        if col not in ['user_id', 'item_id']:
            all_user_item_features[col] = all_user_item_features[col].fillna(0)

    print(f"Создано признаков user-item: {all_user_item_features.shape[1] - 2}")
    print(f"Количество пар user-item: {all_user_item_features.shape[0]}")

    return all_user_item_features

In [21]:
def create_all_user_features(interactions_df, users_meta_df=None, items_meta_df=None, current_date=None):
    """
    Создает все признаки пользователей.
    """
    print("Создание признаков активности пользователей...")
    activity_features = create_user_activity_features(interactions_df, users_meta_df, current_date)

    print("Создание поведенческих признаков...")
    behavioral_features = create_user_behavioral_features(interactions_df, items_meta_df)

    print("Создание временных паттернов...")
    temporal_features = create_user_temporal_patterns(interactions_df)

    print("Создание демографических признаков...")
    demographic_features = create_user_demographic_features(users_meta_df, current_date)

    # Объединяем все признаки
    print("Объединение всех признаков пользователей...")
    user_features = activity_features.copy()

    for features_df in [behavioral_features, temporal_features, demographic_features]:
        if not features_df.empty:
            user_features = user_features.merge(features_df, on='user_id', how='left')

    # Добавляем ранговые признаки
    print("Добавление ранговых признаков...")
    user_features = add_user_rank_features(user_features, interactions_df)

    print(f"Создано признаков пользователей: {user_features.shape[1] - 1}")
    print(f"Количество пользователей: {user_features.shape[0]}")

    return user_features

def add_user_rank_features(user_features, interactions_df):
    """
    Добавляет ранговые признаки.
    """
    user_features_ranked = user_features.copy()

    # 1. Ранг по активности среди всех пользователей
    if 'user_activity_count' in user_features_ranked.columns:
        user_features_ranked['user_activity_rank'] = user_features_ranked['user_activity_count'].rank(pct=True)

    # 2. Ранг по частоте
    if 'user_activity_frequency' in user_features_ranked.columns:
        user_features_ranked['user_frequency_rank'] = user_features_ranked['user_activity_frequency'].rank(pct=True)

    # 3. Ранг по давности (чем меньше дней с последнего визита, тем выше ранг)
    if 'user_recency_days' in user_features_ranked.columns:
        user_features_ranked['user_recency_rank'] = (-user_features_ranked['user_recency_days']).rank(pct=True)

    # 4. Ранг по диверсификации
    if 'user_diversity' in user_features_ranked.columns:
        user_features_ranked['user_diversity_rank'] = user_features_ranked['user_diversity'].rank(pct=True)

    # 5. Квантили активности
    if 'user_activity_count' in user_features_ranked.columns:
        activity_quantiles = pd.qcut(
            user_features_ranked['user_activity_count'],
            q=5,
            labels=['very_low', 'low', 'medium', 'high', 'very_high']
        )
        user_features_ranked['user_activity_quantile'] = activity_quantiles

    return user_features_ranked

In [23]:
from scipy.stats import entropy

In [24]:
# Создаем синтетические данные
np.random.seed(42)

# Взаимодействия
n_interactions = 50000
n_users = 1000
n_items = 500

interactions_data = {
    'user_id': np.random.randint(1, n_users+1, n_interactions),
    'item_id': np.random.randint(1, n_items+1, n_interactions),
    'rating': np.random.choice([1, 2, 3, 4, 5, np.nan], n_interactions, p=[0.05, 0.1, 0.15, 0.3, 0.3, 0.1]),
    'interaction_type': np.random.choice(['click', 'view', 'purchase'], n_interactions, p=[0.6, 0.3, 0.1]),
    'timestamp': pd.date_range(start='2024-01-01', periods=n_interactions, freq='H')
}

interactions_df = pd.DataFrame(interactions_data)

# Метаданные пользователей
users_data = {
    'user_id': range(1, n_users+1),
    'gender': np.random.choice(['M', 'F', 'unknown'], n_users, p=[0.45, 0.45, 0.1]),
    'age': np.random.randint(18, 70, n_users),
    'country': np.random.choice(['US', 'UK', 'DE', 'FR', 'RU'], n_users),
    'registration_date': pd.date_range(start='2022-01-01', periods=n_users, freq='D')
}

users_meta_df = pd.DataFrame(users_data)

# Метаданные объектов
items_data = {
    'item_id': range(1, n_items+1),
    'category': np.random.choice(['Electronics', 'Books', 'Clothing', 'Home', 'Sports'], n_items),
    'brand': np.random.choice(['Brand_A', 'Brand_B', 'Brand_C'], n_items),
    'price': np.random.exponential(100, n_items).round(2)
}

items_meta_df = pd.DataFrame(items_data)

# Создаем признаки
print("СОЗДАНИЕ ПРИЗНАКОВ ПОЛЬЗОВАТЕЛЕЙ")
print("=" * 50)
user_features_df = create_all_user_features(
    interactions_df=interactions_df,
    users_meta_df=users_meta_df,
    items_meta_df=items_meta_df,
    current_date='2024-02-15'
)

print("\nСОЗДАНИЕ ПРИЗНАКОВ USER-ITEM")
print("=" * 50)
user_item_features_df = create_all_user_item_features(
    interactions_df=interactions_df,
    user_features=user_features_df,
    item_features=item_features_df,  # Предположим, что item_features_df уже создан
    items_meta_df=items_meta_df,
    current_date='2024-02-15'
)

print("\nРезультат:")
print(f"Признаков пользователей: {user_features_df.shape[1] - 1}")
print(f"Признаков user-item: {user_item_features_df.shape[1] - 2}")

print("\nПервые 5 строк user-item признаков:")
print(user_item_features_df.head())

print("\nКолонки user-item признаков:")
print(user_item_features_df.columns.tolist()[:20])  # Первые 20 колонок

СОЗДАНИЕ ПРИЗНАКОВ ПОЛЬЗОВАТЕЛЕЙ
Создание признаков активности пользователей...
Создание поведенческих признаков...
Создание временных паттернов...
Создание демографических признаков...


TypeError: unsupported operand type(s) for -: 'DatetimeArray' and 'str'